In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Load BioBart tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')  # Replace with a biomedical BART if available
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


In [12]:
import pandas as pd
from datasets import Dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

# Load the dataset using pandas
df = pd.read_csv('./patient_diagnosis_data.csv')

# Convert pandas DataFrame to Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Load the tokenizer and model (BioBart)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Tokenize the dataset for both 'text' and 'summary'
def tokenize_function(examples):
    inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    summaries = tokenizer(examples['summary'], padding="max_length", truncation=True, max_length=150)
    inputs["labels"] = summaries["input_ids"]  # Set the 'summary' tokenized data as labels
    return inputs

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Since we don't have a validation set
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine-tune the model
trainer.train()


ImportError: cannot import name 'is_threading_enabled' from 'pyarrow.lib' (d:\Marginal utilities\anaconda\Lib\site-packages\pyarrow\lib.cp311-win_amd64.pyd)